In [11]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error, r2_score
import os, fnmatch
from sklearn.cluster import KMeans
from sklearn.preprocessing import PolynomialFeatures 
import seaborn as sns
%matplotlib inline

In [12]:
all_spec_files = []
listOfFiles = os.listdir('./spec')
pattern = "*.spec"
for entry in listOfFiles:
    if fnmatch.fnmatch(entry, pattern):
            all_spec_files.append(entry)

all_fits_files = []
listOfFiles = os.listdir('./specimages')
pattern = "*.fits"
for entry in listOfFiles:
    if fnmatch.fnmatch(entry, pattern):
            all_fits_files.append(entry)
            
            
all_spec_files.sort()
all_fits_files.sort()

In [13]:
def find_id(item_id, id_list):
    for item in id_list:
        if item.split(".fits")[0] == item_id.split(".spec")[0]:
            return True
    return False

In [14]:
all_features = []
for curr_file_name in all_spec_files:
    obj_vector = []
    index = 0
    arr_x = []
    arr_y = []
    f = open("spec/"+curr_file_name,'r')
    if not find_id(curr_file_name, all_fits_files):
        continue
    for line in f:
        if line[0] != "#":
            arr_y.append(float(line.split(" ")[2]))
            arr_x.append(float(line.split(" ")[4].split("\n")[0]))
        elif (line[0] == "#" and line[1] != "#"):
            if(line[0:3:] != "# r"):
                str_arr = line.split(" ")
    #             print(str_arr)
                if(index <= 6):
                    if index == 4 or index == 5 :
                        obj_vector.append(float(str_arr[-1].split("\n")[0]))
                    else:
                        obj_vector.append(str_arr[-1].split("\n")[0])
                else:
                    value = int(float(str_arr[-2]))
                    obj_vector.append(value)

                index += 1
    df = pd.DataFrame({"wl":arr_x, "in":arr_y})
    X = df["wl"]
    Y = df["in"]
    X = X[:, np.newaxis]

    poly = PolynomialFeatures(degree = 8) 
    X_poly = poly.fit_transform(X) 
    
    poly.fit(X_poly, Y) 
    lin2 = LinearRegression() 
    lin2.fit(X_poly, Y) 
    
    for coef in lin2.coef_[1:9:]:
        obj_vector.append(coef)
    
    obj_vector.append("specimages/" + curr_file_name.split(".spec")[0] + ".fits")
    
    all_features.append(obj_vector)
    

In [15]:
result_df = pd.DataFrame(all_features)

In [16]:
result_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,fbs0005,DFBSJ142114.64+425823.6,14:21:14.640,+42:58:23.63,13.86,13.15,OK,101,16,-2.189981e-11,-3.665937e-14,-9.373043e-12,-1.532294e-09,1.024176e-11,-2.280213e-14,2.109259e-17,-6.999758e-21,specimages/fbs0005-DFBSJ142114.64+425823.6.fits
1,fbs0005,DFBSJ142122.01+425921.3,14:21:22.012,+42:59:21.37,12.85,12.20,OK,106,13,-3.792762e-11,-3.647929e-14,-1.208479e-11,-1.865856e-09,1.801383e-11,-4.706942e-14,4.814870e-17,-1.724178e-20,specimages/fbs0005-DFBSJ142122.01+425921.3.fits
2,fbs0005,DFBSJ142125.68+425138.2,14:21:25.688,+42:51:38.29,16.38,14.85,NL,83,17,-1.815388e-11,-1.309881e-14,-3.936086e-12,-6.570811e-10,3.287456e-12,-6.007407e-15,4.796814e-18,-1.419620e-21,specimages/fbs0005-DFBSJ142125.68+425138.2.fits
3,fbs0005,DFBSJ142126.80+425301.4,14:21:26.802,+42:53:01.40,14.45,13.57,NL,120,17,-4.077909e-11,-2.942382e-14,-8.841611e-12,-1.475995e-09,8.681218e-12,-1.793400e-14,1.573814e-17,-5.009265e-21,specimages/fbs0005-DFBSJ142126.80+425301.4.fits
4,fbs0005,DFBSJ142141.43+430014.5,14:21:41.436,+43:00:14.59,16.39,16.47,NL,90,16,-1.024508e-12,-1.714980e-15,-4.384846e-13,-7.168274e-11,5.490072e-13,-1.330955e-15,1.317058e-18,-4.632790e-22,specimages/fbs0005-DFBSJ142141.43+430014.5.fits


In [17]:
result_df.shape

(770, 18)

In [18]:
result_df.rename({0: "plate",
                 1: "object_id",
                 2: "ra",
                 3: "dec",
                 4: "mag_B",
                 5: "mag_R",
                 6: "spectrum_class",
                 7: "spectrum_length",
                 8: "red_bord_position",
                 17: "fits_path",}, inplace=True, axis=1)

In [19]:
# result_df.to_csv("astro_objects.csv")
result_df.head()

,plate,object_id,ra,dec,mag_B,mag_R,spectrum_class,spectrum_length,red_bord_position,9,10,11,12,13,14,15,16,fits_path
0,fbs0005,DFBSJ142114.64+425823.6,14:21:14.640,+42:58:23.63,13.86,13.15,OK,101,16,-2.189981e-11,-3.665937e-14,-9.373043e-12,-1.532294e-09,1.024176e-11,-2.280213e-14,2.109259e-17,-6.999758e-21,specimages/fbs0005-DFBSJ142114.64+425823.6.fits
1,fbs0005,DFBSJ142122.01+425921.3,14:21:22.012,+42:59:21.37,12.85,12.20,OK,106,13,-3.792762e-11,-3.647929e-14,-1.208479e-11,-1.865856e-09,1.801383e-11,-4.706942e-14,4.814870e-17,-1.724178e-20,specimages/fbs0005-DFBSJ142122.01+425921.3.fits
2,fbs0005,DFBSJ142125.68+425138.2,14:21:25.688,+42:51:38.29,16.38,14.85,NL,83,17,-1.815388e-11,-1.309881e-14,-3.936086e-12,-6.570811e-10,3.287456e-12,-6.007407e-15,4.796814e-18,-1.419620e-21,specimages/fbs0005-DFBSJ142125.68+425138.2.fits
3,fbs0005,DFBSJ142126.80+425301.4,14:21:26.802,+42:53:01.40,14.45,13.57,NL,120,17,-4.077909e-11,-2.942382e-14,-8.841611e-12,-1.475995e-09,8.681218e-12,-1.793400e-14,1.573814e-17,-5.009265e-21,specimages/fbs0005-DFBSJ142126.80+425301.4.fits
4,fbs0005,DFBSJ142141.43+430014.5,14:21:41.436,+43:00:14.59,16.39,16.47,NL,90,16,-1.024508e-12,-1.714980e-15,-4.384846e-13,-7.168274e-11,5.490072e-13,-1.330955e-15,1.317058e-18,-4.632790e-22,specimages/fbs0005-DFBSJ142141.43+430014.5.fits


In [20]:
result_df.to_csv("astro_objects.csv")